In [16]:
import re
import pandas as pd

In [17]:
LOG_DIR = "../log_junyi_testing_2"

In [18]:
keyStrokeFile = open(LOG_DIR+"/keyStroke.log", "r")
keyStrokeContent = keyStrokeFile.readlines()

In [19]:
# clean up the content
keyStrokeContent = [line for line in keyStrokeContent if line != "\n" and line != "Keystrokes are now being recorded\n"]
keyStrokeContentRaw = "".join(keyStrokeContent)

In [20]:
def splitByMilestone(inputList):    
    copy = False
    buffer = ""
    splitByMilestone = []
    for line in keyStrokeContent:
        if "---5min KeyLogger milestone" in line:
            copy = True
            splitByMilestone.append(buffer)
            buffer = ""
        if copy:
            buffer += line
    return splitByMilestone

In [21]:
inputByMilestone = splitByMilestone(keyStrokeContent)

In [22]:
def countCmdTabCombo(input):
    pattern = re.compile("\[left-cmd\]\[tab\]")
    count = len(pattern.findall(input))
    return count

In [23]:
print countCmdTabCombo(inputByMilestone[1])

0


In [24]:
def countCmdNumber(input):
    pattern = re.compile("\[left-cmd\]")
    count = len(pattern.findall(input))
    return count

In [25]:
print countCmdNumber(inputByMilestone[1])

18


In [26]:
def calAverageTypingSpeed(input):
    copy = False
    buffer = ""
    splitByMilestone = []
    for line in input.split("\n")[1:]:
        if "***5s KeyLogger Milestone***" in line:
            copy = True
            splitByMilestone.append(buffer)
            buffer = ""
            continue
        if copy:
            buffer += line
    typingSpeed = []
    for line in splitByMilestone:
        if line == '':
            continue
        pattern = re.compile('\[.*?\]', re.IGNORECASE)
        cleanedLine, commandNum = pattern.subn("", line)
        keyNum = len(cleanedLine) + commandNum
        typingSpeed.append(keyNum * 1.0 / 5)
    if len(typingSpeed) == 0:
        return 0
    return round(reduce(lambda x, y: x + y, typingSpeed) / len(typingSpeed), 2)

In [27]:
averageTypingSpeed = calAverageTypingSpeed(inputByMilestone[1])
print averageTypingSpeed

1.55


In [28]:
cmdTabComboList = []
cmdNumberList = []
typingSpeedList = []
for interval in inputByMilestone:
    print interval.split("\n")[0]
    cmdTabComboList.append(countCmdTabCombo(interval))
    print "# of CMD TAB COMBO: " + str(countCmdTabCombo(interval))
    cmdNumberList.append(countCmdNumber(interval))
    print "# of CMD: " + str(countCmdNumber(interval))
    typingSpeedList.append(calAverageTypingSpeed(interval))
    print "Average Typing Speed (key/second): " + str(calAverageTypingSpeed(interval)) 
    print "\n"
    


# of CMD TAB COMBO: 0
# of CMD: 0
Average Typing Speed (key/second): 0


---5min KeyLogger milestone--- 2018-12-03-13:10:53
# of CMD TAB COMBO: 0
# of CMD: 18
Average Typing Speed (key/second): 1.55


---5min KeyLogger milestone--- 2018-12-03-13:15:54
# of CMD TAB COMBO: 1
# of CMD: 20
Average Typing Speed (key/second): 2.21


---5min KeyLogger milestone--- 2018-12-03-13:20:54
# of CMD TAB COMBO: 0
# of CMD: 20
Average Typing Speed (key/second): 2.1


---5min KeyLogger milestone--- 2018-12-03-13:25:55
# of CMD TAB COMBO: 1
# of CMD: 44
Average Typing Speed (key/second): 2.46


---5min KeyLogger milestone--- 2018-12-03-13:30:55
# of CMD TAB COMBO: 0
# of CMD: 2
Average Typing Speed (key/second): 0


---5min KeyLogger milestone--- 2018-12-03-13:35:55
# of CMD TAB COMBO: 0
# of CMD: 0
Average Typing Speed (key/second): 0


---5min KeyLogger milestone--- 2018-12-03-13:40:56
# of CMD TAB COMBO: 0
# of CMD: 0
Average Typing Speed (key/second): 0


---5min KeyLogger milestone--- 2018-12-03-1

In [29]:
len(inputByMilestone)

43

In [30]:
df1 = pd.DataFrame(cmdTabComboList, columns=['Num of cmd tab combo'])
df1.to_csv("cmd_tab_combo_monitor.csv", index=False, sep=' ')
df2 = pd.DataFrame(cmdNumberList, columns=['Num of cmd'])
df2.to_csv("cmd_number_monitor.csv", index=False, sep=' ')
df3 = pd.DataFrame(typingSpeedList, columns=['Average Typing Speed'])
df3.to_csv("typing_speed_monitor.csv", index=False, sep=' ')